In [2]:
import json
import pandas as pd
import numpy as np
from FlagEmbedding import BGEM3FlagModel
from tqdm.auto import tqdm

/home/student/aboba/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
descriptions = json.load(open('/home/student/katya/tbank-ml-camp-sirius/data/labels_with_description.json'))
descriptions = {' '.join(label.split('_')): descriptions[label] for label in list(descriptions)}
labels = list(descriptions)

In [4]:
model = BGEM3FlagModel('BAAI/bge-m3', use_bfp16=True) 

Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 150693.56it/s]


In [5]:
n = len(descriptions)
pairs = []
nearest_labels = {}

for i in tqdm(range(n)):
    embeddings_1 = model.encode(descriptions[labels[i]])['dense_vecs']
    cur_nearest = []
    max_similarity = 0
    for j in range(n):
        if i != j:
            embeddings_2 = model.encode(descriptions[labels[j]])['dense_vecs']
            similarity = embeddings_1 @ embeddings_2.T
            if similarity >= max_similarity:
                cur_nearest.append(labels[j])
                max_similarity = similarity
                if len(cur_nearest) > 3:
                    cur_nearest.pop(0)
                
    nearest_labels[labels[i]] = cur_nearest

100%|██████████| 150/150 [16:25<00:00,  6.57s/it]


In [6]:
nearest_labels

{'translate': ['transfer', 'timer', 'definition'],
 'transfer': ['payday', 'application status', 'balance'],
 'timer': ['time', 'reminder update', 'alarm'],
 'definition': ['translate'],
 'meaning of life': ['fun fact', 'application status', 'shopping list update'],
 'insurance change': ['travel alert', 'oil change how', 'rollover 401k'],
 'find phone': ['definition', 'sync device', 'report lost card'],
 'travel alert': ['insurance change', 'fun fact', 'distance'],
 'pto request': ['timer', 'pto request status', 'pto balance'],
 'improve credit score': ['application status',
  'credit score',
  'credit limit change'],
 'fun fact': ['translate'],
 'change language': ['translate', 'change user name'],
 'payday': ['pto request', 'application status', 'balance'],
 'replacement card duration': ['credit limit change',
  'redeem rewards',
  'expiration date'],
 'time': ['translate', 'distance', 'timezone'],
 'application status': ['transfer', 'shopping list update', 'balance'],
 'flight statu

In [7]:
with open('/home/student/katya/tbank-ml-camp-sirius/data/labels_with_description.json', "w") as f:
    json.dump(descriptions, f, indent=4)
with open('/home/student/katya/tbank-ml-camp-sirius/data/nearest_labels.json', "w") as f:
    json.dump(nearest_labels, f, indent=4)